# Practice Session AI: Decision Tree <a class="tocSkip">

In [ ]:
# vul in

print("Naam:", "Kennes")
print("Voornaam:", "Dries")
print("S-nummer:", "R0486630")
print("Richting:", "ICT")

# druk <ctrl> + <enter>

<img src="https://i.imgur.com/kTl5dQa.jpg" alt="panda in tree" width=500/>

In deze labosessie gaan we zelf één van de *machine learning models* uit de vorige notebook implementeren, namelijk de *Decision Tree*. Net zoals in de vorige sessies levert de *Pandas* library hier de basisstructuren waarin de data wordt voorgesteld. Om de werking van het uitgewerkte algoritme te testen, zullen we dit evalueren op de vertrouwde wijn-dataset.

## Imports and Loading

Om te veel herhaling te voorkomen, wordt de code voor deze stappen gewoon meegegeven. Net zoals vorige week gaan we werken met een ``ModelFrame`` uit Pandas_ML, omwille van de handige ``data`` (*features*) en ``target`` (*target*) accessoren.

In [ ]:
import numpy as np
import pandas as pd
import pandas_ml as pdml

from tqdm import tqdm        ## process bar tool (optional)

In [ ]:
col_names = ['type', 'alcohol', 'malic_acid', 'ash', 'alkalinity', 'magnesium', 'total_phenols', 'flavonoids', 'nonflavonoid_phenols', 'proanthocyanins', 'color_intensity', 'color_hue', 'OD280', 'proline']
df = pd.read_csv("data/wine_orig.csv", names=col_names)

In [ ]:
df.head()

In [ ]:
df_wine = pdml.ModelFrame(df, target='type')

In [ ]:
train_wine, test_wine = df_wine.model_selection.train_test_split(test_size=0.3, random_state=0)
print("Length of training set: {} ({}% of the samples)".format(len(train_wine), len(train_wine)/len(df_wine)))
print("Length of test set: {} ({}% of the samples)".format(len(test_wine), len(test_wine)/len(df_wine)))

In [ ]:
train_wine.head()

## Building Blocks

### Splitter

Eerst en vooral is er een functie nodig die een ``ModelFrame`` opsplitst in twee delen op basis van de waarde van één van de *features*, en deze twee delen teruggeeft.

In [186]:
def split_on_attribute(df_node, attribute, value):
    # <, >=
    # print('split_on_attribute:', attribute, value)
    return df_node.loc[df_node[attribute] < value], df_node.loc[df_node[attribute] >= value]

#split_on_attribute(df_wine, 'alcohol', 13)

### Cost function

De split-functie gaat bij het opstellen van de tree opgeroepen worden wanneer er een *node* binair moet worden gesplitst. Uiteraard is het op voorhand nogal moeilijk te weten welke *feature* en bijbehorende *value* moeten gebruikt worden om tot een zo goed mogelijke split te komen waarbij de klassen zo volledig mogelijk van elkaar te scheiden. Om dit te testen maken we gebruik van enkele concepten uit de informatietheorie, waardoor we tot een *cost function* kunnen komen. 

#### Information Gain and Entropy

*Information gain* is een manier om uit te drukken hoeveel onzekerheid er wordt verloren bij het opsplitsen van de data in een *node*. Deze onzekerheid wordt uitgedrukt in de vorm van *entropy* (eenheid: *bits*), beschreven in de onderstaande formule.

\begin{equation*}
H(X) = - \sum \limits_{i=1}^{n} p(x_i) \log_2 p(x_i)
\end{equation*}

Om de *information gain* tussen een *node* en zijn *children* te maximaliseren, is het voldoende om de split te kiezen waarbij de *entropy* van de *children* minimaal is.

In [187]:
def calc_px(df_node):
    return [df_node.loc[df['type'] == x].size/df_node.size for x in df_node['type'].unique()]

def entropy(df_node):
    px = calc_px(df_node)
    return -sum(px * np.log2(px))

In [188]:
# test entropy
entropy(train_wine)

1.5709478285303144

#### Gini Impurity

Een andere manier om de split te optimaliseren is de *Gini impurity*. Deze is een maat voor de kans dat elementen van een bepaalde klasse in een *node* gemisclassificeerd worden. 

\begin{equation*}
I_G(X) = 1 - \sum \limits_{i=1}^{n} p(x_i)^2
\end{equation*}

Uiteraard willen we dit aantal misclassificaties zo laag mogelijk houden, dus voor een zo goed mogelijke split hebben we graag een minimale *Gini impurity* bij de *children* van de *node*.

In [ ]:
def gini_index(df_node):
    return 1 - sum(x*x for x in calc_px(df_node))

In [190]:
# test Gini impurity
gini_index(train_wine)

0.6601196670135275

#### Weighted Cost

Om de *costs* van de *children* van een *node* in één getal te kunnen weergeven, berekenen we het gewogen gemiddelde van de *splitting cost* voor zowel *entropy* als *Gini impurity* op de volgende manier:

\begin{equation*}
\overline{cost}(N, L, R) = P(L \vert N) \cdot cost(L) + P(R \vert N) \cdot cost(R)
\end{equation*}

In [191]:
def weighted_cost(df_node, node_left, node_right, cost_func):
    return node_left.size / df_node.size * cost_func(node_left) + \
           node_right.size / df_node.size * cost_func(node_right)

In [192]:
# test weighted cost with entropy
weighted_cost(train_wine, *split_on_attribute(train_wine, 'alkalinity', 16), entropy)

1.4577427356053494

In [193]:
# test weighted cost with Gini impurity
weighted_cost(train_wine, *split_on_attribute(train_wine, 'alkalinity', 16), gini_index)

0.6118637247669506

### Apply Split

Nu hoeven we enkel nog een functie te schrijven die over alle waarden van alle *features* in de dataset gaat, hierbij de *cost* berekent, en vervolgens *feature* en waarde teruggeeft waarbij de *splitting cost* het laagste is. 

**Let op:** Op de ``target`` kolom mag **nooit** gesplitst worden!

In [194]:
def find_best_split(df_node, cost_func):
    """
    :param df_node: 
    :param cost_func: 
    :return: (cost: float, attribute: str, value:float)
    """
    best_costs = None
    for col in df_node.columns.values:
        if col == 'type':
            continue
        for val in df_node[col].unique():
            cost = weighted_cost(train_wine, *split_on_attribute(train_wine, col, val), cost_func)
            if best_costs is None or cost < best_costs[0]:
                best_costs = (cost, col, val)
    return best_costs

In [195]:
# test best split finder with entropy
find_best_split(train_wine, gini_index)

(0.3919680182549091, 'color_intensity', 3.84)

In [196]:
#test best split finder with Gini impurity
find_best_split(train_wine, entropy)

(0.9045222630160197, 'flavonoids', 1.58)

### Build Tree

Met behulp van de ``find_best_split`` functie is het nu mogelijk om de *decision tree* recursief op te stellen. Het grootste deel van de functies en attributen van de ``TreeNode`` klasse zijn reeds gegeven, enkel de ``split`` functie (waar de recursie gebeurt) dient nog aangevuld te worden.

In [204]:
class TreeNode(object):
    """ 
    Forms the node of a decision tree.
    
    Args:
        level: Level of the node in the tree.
        df_node: ModelFrame containing the data of the node
        cost_func: Function to calculate the splitting cost (entropy or gini_index).
        max_depth: Maximum depth of the tree.
        min_length: Minimum amount of elements that a node has to contain to be considered splittable.
    """
    counter = 0
    def __init__(self, level, df_node, cost_func, max_depth, min_length, **kwargs):
        self._id = type(self).counter
        type(self).counter += 1
        self.level = level
        self.df_node = df_node
        self.cost_func = cost_func
        self.max_depth = max_depth
        self.min_length = min_length
        self.cost = self.cost_func(self.df_node)
        self.split_attr = None
        self.split_value = None
        self.left = None
        self.right = None
    
    @property
    def record_amt(self):
        """ 
        Returns the amount of data elements in this node's ModelFrame.
        """
        return len(self.df_node)
    
    @property
    def has_children(self):
        """ 
        Check if the node has any children.
        """
        return self.left is not None and self.right is not None
    
    @property
    def class_distribution(self):
        """
        Gives a dict containing the classes as keys and the amount of elements per class as values.
        """
        return self.df_node.target.value_counts().to_dict()
    
    @property
    def category(self):
        """
        Returns the dominant category of the elements in this node.
        """
        return max(self.class_distribution, key=lambda key: self.class_distribution[key])
    
    def add_left(self, node):
        """
        Add another node to this node as left child.
        """
        self.left = node
        
    def add_right(self, node):
        """
        Add another node to this node as right child.
        """
        self.right = node
        
    def split(self):
        """
        Split the node into two children.
        """
        if self.df_node.size < self.min_length:
            print('Leaf', self.df_node.size, '<', self.min_length)
            return
        if self.level >= self.max_depth:
            print('Leaf', self.level.size, '>=', self.max_depth)
            return
        if self.df_node['type'].unique() <= 1:
            print('Leaf pure')
            return
        
        print('Split node #', self._id)
        _, self.split_attr, self.split_value = find_best_split(self.df_node, self.cost_func)
        self.left, self.right = split_on_attribute(self.df_node, self.df_node, self.cost_func)
        print('Split on ', self.split_attr, self.split_value, self.left.size, self.right.size)
        self.left = TreeNode(self.level + 1, self.level, self.cost_func, self.max_depth, self.min_length)
        self.right = TreeNode(self.level + 1, self.right, self.cost_func, self.max_depth, self.min_length)
        self.left.split()
        self.right.split()


In [205]:
def build_tree(train_set, cost_func, max_depth, min_size):
    TreeNode.counter = 0
    root = TreeNode(0, train_set, cost_func, max_depth, min_size)
    root.split()
    return root    

### Print Tree

Om te kijken hoe het getrainde model eruit ziet, wordt er een (rudimentaire) functie meegegeven waarmee de boom recursief kan geprint worden. We moeten er wel voor zorgen dat de juiste velden op de juiste manier geupdated worden in de ``split`` functie van hierboven, anders krijgen we waarschijnlijk een foutboodschap.

In [ ]:
def print_tree(node, sign='<'):
    if node.has_children:
        print(node._id, node.level * "  ", 
              node.split_attr, sign, node.split_value, '---', 
              node.cost, '---', node.class_distribution, '---', node.category)
    else:
        print(node._id, node.level * "  ", sign, "---", 
              node.cost, '---', node.class_distribution, '---', node.category)
    if node.left is not None:
        print_tree(node.left, sign="<")
    if node.right is not None:
        print_tree(node.right, sign=">=")

### Make Predictions

Uiteraard willen we het model niet alleen maar kunnen gebruiken om de trainingsdata te omschrijven (*description*), maar ook om te voorspellen hoe nieuwe data met een ongekend label geclassificeerd wordt door de boom (*prediction*). Hiervoor maken we opnieuw gebruik van recursie: we vertrekken in de *root* en vergelijken ``split_attr`` en ``split_value`` met de testdata. Op basis van deze vergelijking gaan we verder in één van de twee *children* van de *node*, totdat we een *node* bereiken die geen *children* meer heeft. De ``category`` van deze *node* is dan de uitkomst van onze voorspelling.

In [207]:
def predict(data, node):
    raise NotImplementedError("Fill in the code!")

### Evaluate

In [208]:
def evaluate(tree, test_set):
    results = []
    for row in test_wine.itertuples():
        data = row._asdict()
        results.append((data['type'], predict(data, tree)))
    print([f"{i}: {res[0]} {res[1]}" for i, res in enumerate(results)])
    return pd.DataFrame(results, columns=['target','predicted'])

### Calculate Accuracy

In [209]:
def accuracy(tree, test_set):
    df_results = evaluate(tree, test_wine)
    return (df_results.target == df_results.predicted).sum()/len(df_results)

## Testing the Trees

### Using Entropy

In [210]:
tree_entropy = build_tree(train_wine, entropy, 8, 1)
print_tree(tree_entropy)
print("Accuracy: {:4.3f}".format(accuracy(tree_entropy, test_wine)))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

### Using Gini Impurity

In [211]:
tree_gini = build_tree(train_wine, gini_index, 8, 1)
print_tree(tree_gini)
print("Accuracy: {:4.3f}".format(accuracy(tree_gini, test_wine)))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()